## Информация о пользователях, подписанных на сообщества рейв-исполнителей в ВК

Для исследования я скачала данные о пользователях, подписанных на официальные сообщества рейв-исполнителей в ВК.  
Мною были выбраны группы следующих исполнителей: DK, GSPD, Lida, CMH.
Я исследовала 

### Ссылки на сообщества:  
DK: https://vk.com/novaplaycsgo?from=search (id: 113882101)  
GSPD: https://vk.com/gspdpage?from=search (id: 36516670)  
Lida: https://vk.com/lidamudota?from=search (id: 147725517)  
CMH: https://vk.com/jettrec?from=search (id: 47061140)  

## Обозначения  
**id** - id пользователя  
**bdate** - дата рождения  
**city** - город  
**sex** - пол  
**first_name** - имя  
**last_name** - фамилия  
**can_access_closed** - имеет ли тот, кто запрашивает данные (составитель датасета), доступ к информации профиля, даже если он закрыт.  
-   Если  `can_access_closed = True, составитель датасета может видеть данные профиля, даже если он закрыт (например, если они друзья или имеют специальные права доступа).
-   Если  `can_access_closed = False, составитель датасета не имеет доступа к данным профиля, если он закрыт.

**is_closed** - открытый или закрытый профиль (**True** - закрытый; **False** - открытый)  
- Если is_closed = True, профиль закрыт, и доступ к нему ограничен (например, только друзья могут видеть информацию).
- Если is_closed = False, профиль открыт, и его данные доступны для просмотра всем пользователям.  

**deactivated** - деактивированный или активный профиль (**True** - деактивированный; **False** - активный)  
**home_town** - родной город  
**artist** - исполнитель, на чьё сообщество подписан пользователь  
**year_bdate** - год рождения  
**age** - возраст

In [1]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нащего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

In [74]:
# Проверяю, что всё работает

# мой номер странички
my_user_id = 231770150
# версия используемого API
version = '5.103' 
# подгружаю токен
with open('token(1).txt') as f:
    token = f.read()

In [64]:
url = "https://api.vk.com/method/users.get?user_ids={user_id}&access_token={token}&v={version}"
url = url.format(user_id=my_user_id, token=token, version=version)

In [65]:
response = requests.get(url) 
response

<Response [200]>

In [66]:
name = response.json()
name

{'response': [{'id': 231770150,
   'first_name': 'Настя',
   'last_name': 'Строганова',
   'can_access_closed': True,
   'is_closed': False}]}

In [79]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    return response

## Соц-дем пользователей

In [68]:
# Основные параметры, которые нужно получить
fields = 'bdate, city, home_town, sex'

Делаю словарь из списка сообществ

In [83]:
def vk_download1(method, parameters):
    url = 'https://api.vk.com/method/' + method + \
'?' + parameters + '&v=' + version + '&access_token=' + token
    
    response = requests.get(url)
    return response.json()['response']

In [98]:
group_id = '-36516670'

wall = vk_download1('wall.get','owner_id=' + group_id)
wall.keys()

dict_keys(['count', 'items'])

In [93]:
def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей
        
        group_id: string
            идентификатор группы (ссылка)
    """
    
    # Узнаём число запросов, которое надо сделать 
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']
    
    # выясняем, сколько запросов нам понадобиться
    n = int(np.ceil(count/1000))  
    
    # вектор, где мы будем хранить id пользователей
    members = []     
    
    for i in tqdm_notebook(range(n)): 
        # при помощи метода groups.getMembers получаем пользователей группы
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        members.extend(current_members['response']['items'])
        # перед следующим запросом немножко подождем
        time.sleep(0.4)
        
    return members

In [101]:
gspd_members = getGroupMembers('36516670')

C:\Users\Strog\AppData\Local\Temp\ipykernel_24928\1341403498.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/366 [00:00<?, ?it/s]

In [102]:
len(gspd_members)

365398

In [118]:
GSPD_members = gspd_members

In [109]:
DK_members = getGroupMembers('113882101')

C:\Users\Strog\AppData\Local\Temp\ipykernel_24928\1341403498.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/387 [00:00<?, ?it/s]

In [110]:
len(DK_members)

386732

In [111]:
Lida_members = getGroupMembers('147725517')

C:\Users\Strog\AppData\Local\Temp\ipykernel_24928\1341403498.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/344 [00:00<?, ?it/s]

In [112]:
len(Lida_members)

343288

In [113]:
CMH_members = getGroupMembers('47061140')

C:\Users\Strog\AppData\Local\Temp\ipykernel_24928\1341403498.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/323 [00:00<?, ?it/s]

In [114]:
len(CMH_members)

322376

In [119]:
rave = {
     'DK' : DK_members,
     'GSPD' : GSPD_members,
     'Lida' : Lida_members,
     'CMH' : CMH_members
    }

In [121]:
for singer, people in rave.items():
    print("Группа исполнителя: {}, число подписчиков: {}".format(singer, len(people)))

Группа исполнителя: DK, число подписчиков: 386732
Группа исполнителя: GSPD, число подписчиков: 365398
Группа исполнителя: Lida, число подписчиков: 343288
Группа исполнителя: CMH, число подписчиков: 322376


In [123]:
# объединяем всех подписчиков в один лист
unique_people = sum(rave.values(), [])
# берем только уникальных людей
unique_people = list(set(unique_people))

len(unique_people)

1056028

In [124]:
round(1056028/100/60/60, 2)

2.93

In [125]:
cmhshniki = rave['CMH'][:10]
# объединяем лист с int в строку
cmhshniki = ','.join([str(user) for user in cmhshniki])

In [126]:
vkDownload('users.get', 'user_ids={}&fields={}'.format(cmhshniki, fields))

{'response': [{'id': 1018,
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'sex': 2,
   'first_name': 'Ярик',
   'last_name': 'Бурим',
   'can_access_closed': False,
   'is_closed': True},
  {'id': 1490,
   'home_town': '',
   'sex': 2,
   'first_name': 'Poslan',
   'last_name': 'Bogom',
   'can_access_closed': True,
   'is_closed': False},
  {'id': 1775,
   'bdate': '25.3',
   'city': {'id': 1, 'title': 'Москва'},
   'sex': 2,
   'first_name': 'Миша',
   'last_name': 'Поленов',
   'can_access_closed': True,
   'is_closed': False},
  {'id': 2896,
   'city': {'id': 1, 'title': 'Москва'},
   'sex': 1,
   'deactivated': 'deleted',
   'first_name': 'DELETED',
   'last_name': '',
   'can_access_closed': True,
   'is_closed': False},
  {'id': 3450,
   'bdate': '11.12',
   'city': {'id': 1, 'title': 'Москва'},
   'sex': 2,
   'first_name': 'Евгений',
   'last_name': 'Литвинов',
   'can_access_closed': True,
   'is_closed': False},
  {'id': 4041,
   'bdate': '1.7.1995',
   'city': {'id': 

In [127]:
def getUserInformation():
    """
        Возвращает словарь с выгруженными социально-демографическими данными 
        по всем пользователям, подписанным на группы
    """
    
    # инициализируем пустой словарь, где будут хранитсья пользователи с их соцдемом
    rave_information = {}
    # проходимся по всем кандидатам
    for singer, people in rave.items():
        # текущий список пользователей с соц-демом
        current_info = []
        #посчитаем число пользователей для группы
        count = len(people)
        # Выяснили число запросов
        n = int(np.ceil(count/500))  
        
        for i in tqdm_notebook(range(n)):
            # берем текущий срез подписчиков
            ids = people[i*500:(i+1)*500]
            ids = ','.join([str(user) for user in ids])
            # выгружаем их и берем данные
            info = vkDownload('users.get','user_ids={}&fields={}'.format(ids, fields))
            info = info['response']
            # записываем в текущий лист
            current_info.extend(info)
            # ждем перед следующим запросом
            time.sleep(0.4)
        
        # записываем в финальный словарь
        rave_information[singer] = current_info
    
    return rave_information

In [128]:
rave_information = getUserInformation()

C:\Users\Strog\AppData\Local\Temp\ipykernel_24928\756148831.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/774 [00:00<?, ?it/s]

  0%|          | 0/731 [00:00<?, ?it/s]

  0%|          | 0/687 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

In [129]:
#спрячу данные в дамп pickle
with open('rave_information', 'wb') as f:
     pickle.dump(rave_information, f)

#если нужно открыть дамп из pickle
#with open('university_people_information', 'rb') as f:
    #candidate_people_information = pickle.load(f)

In [173]:
# инициализирую датафрейм
rave_information_df = pd.DataFrame()

# иду по всем кандидатам и соц-дему их подписчиков
for singer, people in rave_information.items():
    # преобразую в словарь в датафрейм
    df = pd.DataFrame(people)
    # добавляю столбец с городом
    df['city'] = df.city.apply(lambda x: x['title'] if x is not np.nan else np.nan)

    # Создаем словарь user_id -> artist
    user_to_singer = {}
    for singer, people in rave.items():
        for person in people:
            user_to_singer[person] = singer
    
    # Добавляем столбец artist
    df['artist'] = df['id'].map(user_to_singer)
    
    # добавляем преобразованный датафрейм к финальному
    rave_information_df = pd.concat([rave_information_df, df], ignore_index=True)

In [174]:
rave_information_df

,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist
0,619,31.12,Stockholm,2,Алексей,Бардаш,False,True,NaN,NaN,DK
1,1011,1.10,Санкт-Петербург,2,Максим,Петренчук,True,False,NaN,NaN,DK
2,2136,23.10,Рязань,2,Ярослав,Техник,True,False,NaN,NaN,GSPD
3,2896,NaN,Москва,1,DELETED,,True,False,deleted,NaN,CMH
4,5251,6.9.1987,NaN,2,Кирилл,Мищенко,True,False,NaN,NaN,DK
...,...,...,...,...,...,...,...,...,...,...,...
1417789,1035923981,6.3.2011,NaN,1,Женя,Тушенцова,True,False,NaN,,CMH
1417790,1036131415,16.6.2001,NaN,1,Шуша,Блиновская,True,False,NaN,,CMH
1417791,1036180158,1.8.2005,Боровичи,2,Фёдор,Ульянов,True,False,NaN,,CMH
1417792,1036245822,16.3.1993,Саров,1,Ksusha,Arimova,True,False,NaN,,CMH


In [175]:
rave_information_df.shape

(1417794, 11)

In [176]:
rave_information_df.to_csv('rurave_people.csv', sep=',')

In [177]:
rave_information_df = pd.read_csv('rurave_people.csv')

In [178]:
rave_information_df

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist
0,0,619,31.12,Stockholm,2,Алексей,Бардаш,False,True,NaN,NaN,DK
1,1,1011,1.10,Санкт-Петербург,2,Максим,Петренчук,True,False,NaN,NaN,DK
2,2,2136,23.10,Рязань,2,Ярослав,Техник,True,False,NaN,NaN,GSPD
3,3,2896,NaN,Москва,1,DELETED,NaN,True,False,deleted,NaN,CMH
4,4,5251,6.9.1987,NaN,2,Кирилл,Мищенко,True,False,NaN,NaN,DK
...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,1,Женя,Тушенцова,True,False,NaN,NaN,CMH
1417790,1417790,1036131415,16.6.2001,NaN,1,Шуша,Блиновская,True,False,NaN,NaN,CMH
1417791,1417791,1036180158,1.8.2005,Боровичи,2,Фёдор,Ульянов,True,False,NaN,NaN,CMH
1417792,1417792,1036245822,16.3.1993,Саров,1,Ksusha,Arimova,True,False,NaN,NaN,CMH


## Преобразование данных

In [179]:
# Меняю числовые значения в столбце 'sex' (пол) на слова
rave_information_df['sex'] = np.where(rave_information_df['sex']==1, 'Female', 'Male')

In [180]:
rave_information_df['sex']

0            Male
1            Male
2            Male
3          Female
4            Male
            ...  
1417789    Female
1417790    Female
1417791      Male
1417792    Female
1417793      Male
Name: sex, Length: 1417794, dtype: object

## Определение возраста пользователя по дате рождения

In [181]:
rave_information_df['bdate'].str.split('.', expand=True)
#агрумент expand=True значит создание новых столбцов из разденного

,0,1,2
0,31,12,None
1,1,10,None
2,23,10,None
3,NaN,NaN,NaN
4,6,9,1987
...,...,...,...
1417789,6,3,2011
1417790,16,6,2001
1417791,1,8,2005
1417792,16,3,1993


In [183]:
df_bdate = rave_information_df

In [184]:
#датафрейм df_ нужен для временного хранения данных о дате рождения в разных столбцах
df_ = df_bdate['bdate'].str.split('.', expand=True)

In [185]:
#объединяем два датарфрейиа по index
df_bdate = pd.concat([df_bdate, df_], axis=1)

In [186]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,0,1,2
0,0,619,31.12,Stockholm,Male,Алексей,Бардаш,False,True,NaN,NaN,DK,31,12,None
1,1,1011,1.10,Санкт-Петербург,Male,Максим,Петренчук,True,False,NaN,NaN,DK,1,10,None
2,2,2136,23.10,Рязань,Male,Ярослав,Техник,True,False,NaN,NaN,GSPD,23,10,None
3,3,2896,NaN,Москва,Female,DELETED,NaN,True,False,deleted,NaN,CMH,NaN,NaN,NaN
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,6,9,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,6,3,2011
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,16,6,2001
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,1,8,2005
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,16,3,1993


In [187]:
df_bdate = df_bdate.rename(columns={0: 'day_bdate', 1: 'month_bdate', 2: 'year_bdate'})

In [188]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,day_bdate,month_bdate,year_bdate
0,0,619,31.12,Stockholm,Male,Алексей,Бардаш,False,True,NaN,NaN,DK,31,12,None
1,1,1011,1.10,Санкт-Петербург,Male,Максим,Петренчук,True,False,NaN,NaN,DK,1,10,None
2,2,2136,23.10,Рязань,Male,Ярослав,Техник,True,False,NaN,NaN,GSPD,23,10,None
3,3,2896,NaN,Москва,Female,DELETED,NaN,True,False,deleted,NaN,CMH,NaN,NaN,NaN
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,6,9,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,6,3,2011
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,16,6,2001
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,1,8,2005
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,16,3,1993


In [189]:
#Фильтрую аудиторию по признаку открытости страницы
df_bdate = df_bdate.loc[((df_bdate['can_access_closed'] == True))]

In [190]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,day_bdate,month_bdate,year_bdate
1,1,1011,1.10,Санкт-Петербург,Male,Максим,Петренчук,True,False,NaN,NaN,DK,1,10,None
2,2,2136,23.10,Рязань,Male,Ярослав,Техник,True,False,NaN,NaN,GSPD,23,10,None
3,3,2896,NaN,Москва,Female,DELETED,NaN,True,False,deleted,NaN,CMH,NaN,NaN,NaN
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,6,9,1987
5,5,9036,24.9,NaN,Male,Александр,Корнев,True,False,NaN,NaN,Lida,24,9,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,6,3,2011
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,16,6,2001
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,1,8,2005
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,16,3,1993


In [191]:
#удаляю столбцы с днем и месяцем рождения, оставляю только год
df_bdate = df_bdate.drop(['day_bdate', 'month_bdate'], axis=1)

In [192]:
df_bdate.isnull().sum()

Unnamed: 0                0
id                        0
bdate                169249
city                 500450
sex                       0
first_name                4
last_name             68846
can_access_closed         0
is_closed                 0
deactivated          942617
home_town            802945
artist                    0
year_bdate           517647
dtype: int64

Удаляю все строки, если в столбце year_bdate NaN

In [193]:
df_bdate = df_bdate.dropna(subset=['year_bdate'])

In [194]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,year_bdate
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,1987
7,7,10621,25.5.2000,NaN,Male,Максим,Долларов,True,False,NaN,NaN,DK,2000
13,13,90729,26.11.2001,NaN,Female,Анастасия,Царёва,True,False,banned,NaN,DK,2001
18,18,112116,25.2.1988,Санкт-Петербург,Female,Ангелина,Крылова,True,False,NaN,Санкт Петербург,CMH,1988
21,21,140797,5.3.1989,Санкт-Петербург,Female,Катюня,Попайя,True,False,NaN,Санкт-Петербург,CMH,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,2011
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,2001
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,2005
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,1993


In [195]:
df_bdate['age'] = 2025 - df_bdate['year_bdate'].astype (int)

In [196]:
df_bdate.dtypes

Unnamed: 0            int64
id                    int64
bdate                object
city                 object
sex                  object
first_name           object
last_name            object
can_access_closed      bool
is_closed              bool
deactivated          object
home_town            object
artist               object
year_bdate           object
age                   int32
dtype: object

In [197]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,year_bdate,age
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,1987,38
7,7,10621,25.5.2000,NaN,Male,Максим,Долларов,True,False,NaN,NaN,DK,2000,25
13,13,90729,26.11.2001,NaN,Female,Анастасия,Царёва,True,False,banned,NaN,DK,2001,24
18,18,112116,25.2.1988,Санкт-Петербург,Female,Ангелина,Крылова,True,False,NaN,Санкт Петербург,CMH,1988,37
21,21,140797,5.3.1989,Санкт-Петербург,Female,Катюня,Попайя,True,False,NaN,Санкт-Петербург,CMH,1989,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,2011,14
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,2001,24
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,2005,20
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,1993,32


Для вычисления возраста нужно от текущего года вычесть год рождения. Для этого переменная year_bdate должна быть числом



In [198]:
df_bdate['year_bdate'] = pd.to_numeric(df_bdate['year_bdate'], errors= 'coerce')

In [199]:
df_bdate.dtypes

Unnamed: 0            int64
id                    int64
bdate                object
city                 object
sex                  object
first_name           object
last_name            object
can_access_closed      bool
is_closed              bool
deactivated          object
home_town            object
artist               object
year_bdate            int64
age                   int32
dtype: object

In [200]:
df_bdate['age'] = 2025 - df_bdate['year_bdate']

In [201]:
df_bdate

,Unnamed: 0,id,bdate,city,sex,first_name,last_name,can_access_closed,is_closed,deactivated,home_town,artist,year_bdate,age
4,4,5251,6.9.1987,NaN,Male,Кирилл,Мищенко,True,False,NaN,NaN,DK,1987,38
7,7,10621,25.5.2000,NaN,Male,Максим,Долларов,True,False,NaN,NaN,DK,2000,25
13,13,90729,26.11.2001,NaN,Female,Анастасия,Царёва,True,False,banned,NaN,DK,2001,24
18,18,112116,25.2.1988,Санкт-Петербург,Female,Ангелина,Крылова,True,False,NaN,Санкт Петербург,CMH,1988,37
21,21,140797,5.3.1989,Санкт-Петербург,Female,Катюня,Попайя,True,False,NaN,Санкт-Петербург,CMH,1989,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417789,1417789,1035923981,6.3.2011,NaN,Female,Женя,Тушенцова,True,False,NaN,NaN,CMH,2011,14
1417790,1417790,1036131415,16.6.2001,NaN,Female,Шуша,Блиновская,True,False,NaN,NaN,CMH,2001,24
1417791,1417791,1036180158,1.8.2005,Боровичи,Male,Фёдор,Ульянов,True,False,NaN,NaN,CMH,2005,20
1417792,1417792,1036245822,16.3.1993,Саров,Female,Ksusha,Arimova,True,False,NaN,NaN,CMH,1993,32


In [202]:
df_bdate['sex']

4            Male
7            Male
13         Female
18         Female
21         Female
            ...  
1417789    Female
1417790    Female
1417791      Male
1417792    Female
1417793      Male
Name: sex, Length: 515590, dtype: object

In [203]:
df_bdate.groupby(
    by=['artist'])[['age']].mean().sort_values(by='age', ascending=False)

,age
artist,
DK,30.444771
CMH,29.775400
GSPD,29.475663
Lida,29.162228


In [204]:
df_bdate.groupby(
    by=['sex'])[['age']].mean().sort_values(by='age', ascending=False)

,age
sex,
Male,30.859649
Female,28.331964


## Откуда аудитория? Анализ городов

In [205]:
rave_information_df['city'].unique()

array(['Stockholm', 'Санкт-Петербург', 'Рязань', ..., 'Сусат',
       'Bay Harbor Islands', 'Думиничи (станция)'], dtype=object)

In [206]:
df['city'].nunique()

6304

In [207]:
df_bdate.groupby(
    by=['city'])[['id']].count().sort_values(by='id', ascending=False).head(100)

,id
city,
Москва,35836
Санкт-Петербург,23663
Екатеринбург,5505
Новосибирск,5061
Краснодар,3718
...,...
Брест,418
Stockholm,413
Абакан,392


In [208]:
df_bdate.to_csv('rave_data.csv', index=False)

In [212]:
import os
path = os.path.join('D:', 'OD_Final_project', 'rave_data.csv')
df_bdate.to_csv(path, index=False)